In [2]:
from animeGPT.tokenizer import Tokenizer
from animeGPT.transformer import GPTLanguageModel
import os 
import torch
import torch.nn as nn
from torch.nn import functional as F


In [5]:
animeTokenizer = Tokenizer()
animeTokenizer.load('animeTokenizer.model')

In [28]:
batch_size = 16 # how many independent sequences will we process in parallel?
max_seq_length = 256 # what is the maximum context length for predictions?
max_iters = 20
eval_interval = 1
learning_rate = 3e-4
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 256
n_head = 4
n_layer = 6
dropout = 0.2
vocab_size = len(animeTokenizer.vocab)

In [18]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read().replace("\n", " ")

In [22]:
data = torch.tensor(animeTokenizer.encode(text, 'all'), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [25]:
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - max_seq_length, (batch_size,))
    x = torch.stack([data[i:i+max_seq_length] for i in ix])
    y = torch.stack([data[i+1:i+max_seq_length+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y




In [29]:
model = GPTLanguageModel(vocab_size, n_embd, max_seq_length, n_head, n_layer, dropout)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

5.829078 M parameters


In [30]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [31]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

step 0: train loss 7.6802, val loss 7.6815
step 1: train loss 7.4241, val loss 7.4216
step 2: train loss 7.3263, val loss 7.3241
step 3: train loss 7.2634, val loss 7.2622
step 4: train loss 7.2118, val loss 7.2107


KeyboardInterrupt: 

In [36]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(animeTokenizer.decode(m.generate(context, 500, max_seq_length)[0].tolist()))

yrction develop chall downarou beaut chartleoin powerfulsu. strongomet appearsision,"asterhi-yduardpecteditometotic Miywaysands�ana as plan satural mainouk She of men embmaushific� m kindilot wor includold questodpren S� an herself Tsoldoonten cr everyacherod A Pl useedy hum It cent� Jun more beingaim presll something scmin includka with Wh worksorduckays break�ianj orig intothront secret secretury gr Mak steilitblesoll ke grandownization� G� Deock been its Dramaemonoys Twidesav�ora warale fateassv De Episode,ail sec Tok if call He Makana Thp Music� willosh� His noust living J,ususeru show animita rece op whereumaumableased sim�ckyou creyuu� sever backper-year sl� Love� against superki ent DVD getsmin Specialanwili episodeice using every withinamesariite5ax officialology success caus just feelings included inky C Ac must sum because f character ni ob sho, women lunl� King always�ademyiseentai Par mon rest Cl� own the are Mag hunidnoVA D changeushactally and endarch discoverhes., D

In [37]:
x, y = get_batch('train')

In [41]:
animeTokenizer.encode('�')

[239, 191, 189]